# Simple Task

In [31]:
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import os, json
from maestro.api import APIClient, Provider
from maestro.api import Dataset, Image, Task, Group
from maestro.schemas import TaskInputs

In [2]:
token="fcad6b3734b74e7e901c58903dc416f8c5029cf224714f518f3f04ac379d7b7a"
host="http://caloba51.lps.ufrj.br:7000"
basepath = os.getcwd()

## Create remote session:

In [5]:
provider = Provider(host=host, token=token)

## create the dataset:

In [6]:
def create_dataset(path, n):
    os.makedirs(path, exist_ok=True)
    for i in range(n):
        with open(f"{path}/{i}.json",'w') as f:
            d={'a':i*10,'b':i*2}
            json.dump(d,f)
            
create_dataset(basepath+"/jobs" , 10)

In [7]:
dataset = Dataset(name="user.joao.pinto.dataset")

In [8]:
dataset.create()

2025-03-24 15:16:04.397 | DEBUG    | maestro.api.rest.dataset:create:104 - Received dataset_id: 6468a74db763


'6468a74db763'

In [9]:
dataset.upload(basepath+"/jobs",as_link=True)

uploading file...: 100%|████████████████████████████████████████████| 10/10 [00:00<00:00, 10.52it/s]


True

In [10]:
dataset.print()

Dataset name : user.joao.pinto.dataset
dataset id   : 6468a74db763
+------------+----------------------------------+
| filename   | md5                              |
|------------+----------------------------------|
| 8.json     | c5fdd711b31fa54e8d68dda7e28b5a55 |
| 5.json     | 5158084def9716b1861a32e778de5665 |
| 4.json     | 48a462caf5f9f036054c3d5bb30ac195 |
| 1.json     | c2b86c9b4033debbeec4b497a45734c1 |
| 3.json     | 664d74949c77a3bfce4a8ac95244c1f8 |
| 0.json     | 7f9fb369851609ce9cb91404549393f3 |
| 6.json     | 94e517ae388d646047a203ecab9952f0 |
| 2.json     | da03619f5a6b5195ce248f24c0eddd64 |
| 9.json     | eee20f56e7111321d839c75dbaac54c1 |
| 7.json     | 42fd321dda1be4cb510182aab78263bc |
+------------+----------------------------------+


## Create the image:

In [11]:
image = Image("joao.pinto/python3.10", basepath+"/python3.10.sif", as_link=True)

## Create the task:

In [12]:
task = TaskInputs( name="user.joao.pinto.simple_task3", image='joao.pinto/python3.10' )
task.input   = dataset.name
task.command = "python /app.py -j %IN -o %OUT"
task.outputs = {"OUT":'result.json'}
task.envs    = {}
task.partition = "cpu-large"

In [14]:
my_task=provider.create(task)

In [19]:
my_task.print()

+------------------------------+--------------+------------+-----------+-----------+----------+--------+----------+----------+-------------+-----------+
| taskname                     |   registered |   assigned |   pending |   running |   failed |   kill |   killed |   broken |   completed | Status    |
|------------------------------+--------------+------------+-----------+-----------+----------+--------+----------+----------+-------------+-----------|
| user.joao.pinto.simple_task3 |            0 |          0 |         0 |         0 |        0 |      0 |        0 |        0 |          10 | completed |
+------------------------------+--------------+------------+-----------+-----------+----------+--------+----------+----------+-------------+-----------+


In [20]:
provider.print_datasets()

+--------------+------------------------------------------+---------+--------+
| id           | name                                     |   files | type   |
|--------------+------------------------------------------+---------+--------|
| 6468a74db763 | user.joao.pinto.dataset                  |      10 | files  |
| 55f87ba95f53 | joao.pinto/python3.10                    |       1 | image  |
| 4bf61d1215df | user.joao.pinto.simple_task3.result.json |      10 | files  |
+--------------+------------------------------------------+---------+--------+


In [21]:
provider.print_tasks()

+--------------+------------------------------+-----------+
| id           | name                         | type      |
|--------------+------------------------------+-----------|
| 0f789a0f2c04 | user.joao.pinto.simple_task3 | completed |
+--------------+------------------------------+-----------+
